In [26]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `C:\Users\Ferdi`


In [27]:
using HiGHS
using JuMP
using Plots
using XLSX
using DataFrames
using Ipopt
using Complementarity

In [28]:
T=3 #Number of time periods
W=3 #Number of scenarios
probW=[0.2,0.3,0.5] #Probability of scenarios

EGmax=5 #max energy production Mw at VPP
EGmin=1 #min ""

ELmax=2   #max energy consumption
ELmin=0.5 #min ""
EL_floor= 2 # energy must be consumend in T

PdSmax=0.5 #max discharging power
PcSmax=0.3 #max charging power
ndS=0.8 #loss by discharging
ncS=0.8 #loss by charging 
ESmax=1   #max energy storage
ESmin=0.2 #min "" 
E_S0=0.6 #energy storage t=0

PWexcept=[3.3,3.0,3.00] #forcast power production for periods
PW=[2.5 6.0 2.0; #power production at t in w
    4.0 4.0 1.1;
    6.0 3.5 1.5]

yD=[20,80,45] #market price

Utility=[10 20 10; #utility at t in w
    50 20 20;
    10 30 10]
Cost = [5 10 10; #cost of production at t in w
    15 20 15;
    30 15 10]

3×3 Matrix{Int64}:
  5  10  10
 15  20  15
 30  15  10

In [29]:
m3 = Model(HiGHS.Optimizer)


@variable(m3, v[t in 1:T, w in 1:W], Bin)
@variable(m3, E_G[t in 1:T, w in 1:W])
@variable(m3, E_L[t in 1:T, w in 1:W])
@variable(m3, E_S[t in 1:T, w in 1:W])
@variable(m3, Pd_S[t in 1:T, w in 1:W] >= 0)
@variable(m3, Pc_S[t in 1:T, w in 1:W] >= 0)
@variable(m3, P_D[t in 1:T])
@variable(m3, PWcurt[t in 1:T, w in 1:W] >= 0)

@constraint(m3, VPPenergyproductionlow[t in 1:T, w in 1:W], v[t,w]*EGmin <= E_G[t,w])
@constraint(m3, VPPenergyproductionup[t in 1:T, w in 1:W], E_G[t,w] <= v[t,w]*EGmax)

@constraint(m3, loadenergyconsumption[t in 1:T, w in 1:W], ELmin <= E_L[t,w] <= ELmax)
@constraint(m3, floorenergyconsumption[t in 1:T, w in 1:W], EL_floor <= sum(E_L[t,w] for t in 1:T))


@constraint(m3, storagedischarging[t in 1:T, w in 1:W], Pd_S[t,w] <= PdSmax)
@constraint(m3, storagecharging[t in 1:T, w in 1:W], Pc_S[t,w] <= PcSmax)
@constraint(m3, storagelevel1[t in 1:1, w in 1:W], E_S[t,w] == E_S0 + ncS*Pc_S[t,w] - (1/ndS)*Pd_S[t,w])


@constraint(m3, storagelevel[t in 1:T, w in 1:W], E_S[t,w] == E_S[t,w] + ncS*Pc_S[t,w] - (1/ndS)*Pd_S[t,w])
@constraint(m3, storagelevelcapacity[t in 1:T, w in 1:W], ESmin <= E_S[t,w] <=ESmax)
@constraint(m3, PVproductioncurtailment[t in 1:T, w in 1:W], PWcurt[t,w] <= PW[t,w])

@constraint(m3, energybalance[t in 1:T, w in 1:W], E_G[t,w] + PW[t,w] - PWcurt[t,w] + Pd_S[t,w] == E_L[t,w] + Pc_S[T,w] + P_D[t])


3×3 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 energybalance[1,1] : E_G[1,1] - E_L[1,1] + Pd_S[1,1] - Pc_S[3,1] - P_D[1] - PWcurt[1,1] == -2.5  …  energybalance[1,3] : E_G[1,3] - E_L[1,3] + Pd_S[1,3] - Pc_S[3,3] - P_D[1] - PWcurt[1,3] == -2
 energybalance[2,1] : E_G[2,1] - E_L[2,1] + Pd_S[2,1] - Pc_S[3,1] - P_D[2] - PWcurt[2,1] == -4       energybalance[2,3] : E_G[2,3] - E_L[2,3] + Pd_S[2,3] - Pc_S[3,3] - P_D[2] - PWcurt[2,3] == -1.1
 energybalance[3,1] : E_G[3,1] - E_L[3,1] + Pd_S[3,1] - Pc_S[3,1] - P_D[3] - PWcurt[3,1] == -6       energybalance[3,3] : E_G[3,3] - E_L[3,3] + Pd_S[3,3] - Pc_S[3,3] - P_D[3] - PWcurt[3,3] == -1.5

In [30]:
optimize!(m3)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
36 rows, 48 cols, 114 nonzeros
30 rows, 42 cols, 90 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   30 rows
   42 cols (9 binary, 0 integer, 0 implied int., 33 continuous)
   90 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      0
  Dual bound        0
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    0 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.01 (t

In [31]:
echo "# Amo_Bonustask3" >> README.md
  git init
  git add README.md
  git commit -m "first commit"
  git branch -M main
  git remote add origin https://github.com/FerdiTose/Amo_Bonustask3.git
  git push -u origin main

LoadError: syntax: extra token """ after end of expression